In [5]:
import pandas as pd

print(len(pd.read_json('/kovar-vol/kovar/dataset/train.json', lines=True)))
print(len(pd.read_json('/kovar-vol/kovar/dataset/test.json', lines=True)))

11068 + 1216

11068
1216


12284

### Comparing CLIP vs KoCLIP

In [6]:
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(text=["a photo of a cat", "a photo of a dog"], images=image, return_tensors="pt", padding=True)

outputs = model(**inputs)
logits_per_image = outputs.logits_per_image # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities

In [7]:
# processor.tokenizer.add_special_tokens({'sep_token':'[SEP]'})
processor.tokenizer

CLIPTokenizerFast(name_or_path='openai/clip-vit-base-patch32', vocab_size=49408, model_max_length=77, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|startoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True)

In [2]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'pixel_values'])

In [20]:
from transformers import AutoProcessor, AutoModel
import torch
processor = AutoProcessor.from_pretrained("koclip/koclip-base-pt")
model = AutoModel.from_pretrained("koclip/koclip-base-pt")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)


text = ["소파 위에 고양이", "강아지와 강아지 주인", "쳇바퀴를 달리는 햄스터", "자동차"]

inputs = processor(
    text=text,
    images=image, 
    return_tensors="pt", # could also be "pt" 
    padding=True
)

outputs = model(**inputs)
probs = torch.nn.functional.softmax(outputs.logits_per_image)

for idx, prob in sorted(enumerate(*probs), key=lambda x: x[1], reverse=True):
    print(text[idx], prob)

소파 위에 고양이 tensor(0.8265, grad_fn=<UnbindBackward0>)
자동차 tensor(0.1040, grad_fn=<UnbindBackward0>)
쳇바퀴를 달리는 햄스터 tensor(0.0655, grad_fn=<UnbindBackward0>)
강아지와 강아지 주인 tensor(0.0040, grad_fn=<UnbindBackward0>)


/tmp/ipykernel_2364493/4056635452.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = torch.nn.functional.softmax(outputs.logits_per_image)


In [21]:
processor.tokenizer.sep_token

'[SEP]'

In [9]:
inputs['token_type_ids'].shape

torch.Size([4, 7])